<a href="https://colab.research.google.com/github/ach224/Prediction_eligibilite_pret_bancaire/blob/App_gradio_A%C3%AFcha/App_gradio_A%C3%AFcha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interface Gradio

In [ ]:
import gradio as gr

def hello(name):
    return f"Bonjour {name} !"

gr.Interface(fn=hello, inputs="text", outputs="text").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5341d1b469647ff6e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Importation du dataset

In [10]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/DATA SCIENCES/PROJET DATA SCIENCES/loan_prediction.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


# Préparation du dataset + random forest

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import gradio as gr

# =====================================================
# 1. Chargement & préparation des données (même pipeline que le notebook)
# =====================================================

df = pd.read_csv("/content/drive/MyDrive/DATA SCIENCES/PROJET DATA SCIENCES/loan_prediction.csv")

# 1) Imputation des valeurs manquantes
# Numériques
for col in ["LoanAmount", "Loan_Amount_Term"]:
    df[col] = df[col].fillna(df[col].median())

# Catégorielles
for col in ["Gender", "Married", "Dependents", "Self_Employed", "Credit_History"]:
    df[col] = df[col].fillna(df[col].mode()[0])

# 2) Encodage de la cible
df["Loan_Status"] = df["Loan_Status"].map({"Y": 1, "N": 0})

# 3) Conversion en int de certaines colonnes
df["Dependents"] = df["Dependents"].replace("3+", 3).astype(int)
df["CoapplicantIncome"] = df["CoapplicantIncome"].astype(int)
df["LoanAmount"] = df["LoanAmount"].astype(int)
df["Loan_Amount_Term"] = df["Loan_Amount_Term"].astype(int)
df["Credit_History"] = df["Credit_History"].astype(int)

# 4) One-hot encoding
cat_cols = ["Gender", "Married", "Education", "Self_Employed", "Property_Area"]
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# 5) Feature engineering : Income-to-Loan-Ratio
df["Income-to-Loan-Ratio"] = (
    df["ApplicantIncome"] + df["CoapplicantIncome"]
) / df["LoanAmount"]

# Suppression de Loan_ID
df = df.drop(columns=["Loan_ID"])

# 6) Séparation X / y et split
X = df.drop(columns=["Loan_Status"])
y = df["Loan_Status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# On garde la liste des colonnes pour aligner les nouvelles données
FEATURE_COLUMNS = X_train.columns

# 7) Modèle Random Forest (après Feature Engineering)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

# Fontion de prédiction

In [12]:
# =====================================================
# 2. Fonction de prédiction pour Gradio
# =====================================================

def predict_loan(
    gender,
    married,
    dependents,
    education,
    self_employed,
    applicant_income,
    coapplicant_income,
    loan_amount,
    loan_term,
    credit_history,
    property_area,
):
    # Construction d'une ligne de données brute
    row = {
        "gender": gender,
        "Married": married,
        "Dependents": dependents,
        "education": education,
        "self_employed": self_employed,
        "applicant_income": applicant_income,
        "coapplicant_income": coapplicant_income,
        "loan_amount": loan_amount,
        "loan_term": loan_term,
        "credit_history": credit_history,
        "property_area": property_area,
    }

    df_input = pd.DataFrame([row])

    # Même prétraitement que pour le train

    # Dependents : "3+" -> 3 puis int
    df_input["Dependents"] = df_input["Dependents"].replace("3+", 3).astype(int)

    # Colonnes numériques en int
    num_cols = [
        "ApplicantIncome",
        "CoapplicantIncome",
        "LoanAmount",
        "Loan_Amount_Term",
        "Credit_History",
    ]
    df_input[num_cols] = df_input[num_cols].astype(int)

    # One-hot encoding sur les mêmes colonnes
    df_input = pd.get_dummies(df_input, columns=cat_cols, drop_first=True)

    # Feature engineering
    df_input["Income-to-Loan-Ratio"] = (
        df_input["ApplicantIncome"] + df_input["CoapplicantIncome"]
    ) / df_input["LoanAmount"]

    # Alignement des colonnes avec le modèle (les colonnes manquantes sont mises à 0)
    df_input = df_input.reindex(columns=FEATURE_COLUMNS, fill_value=0)

    # Prédiction
    proba = rf_model.predict_proba(df_input)[0, 1]
    pred = rf_model.predict(df_input)[0]

    decision = "Prêt accepté ✅" if pred == 1 else "Prêt refusé ❌"
    proba_txt = f"Probabilité d'acceptation : {proba:.2%}"

    return decision, proba_txt


# Interface Gradio

In [ ]:
# =====================================================
# 3. Interface Gradio
# =====================================================

inputs = [
    gr.Radio(["Homme", "Femme"], label="Quel est votre genre ?"),
    gr.Radio(["Oui", "Non"], label="Êtes-vous marié ?"),
    gr.Radio(["0", "1", "2", "3+"], label="Combien de personnes sont rattachés à votre foyer ?"),
    gr.Radio(["Diplomé", "Non diplomé"], label="Êtes-vous diplomé ?"),
    gr.Radio(["Oui", "Non"], label="Êtes-vous auto-entrepreneur ?"),
    gr.Number(label="Quels sont vos revenus mensuels (en €) ?"),
    gr.Number(label="Quels sont les revenus mensuels de votre partenaire (en €) ?"),
    gr.Number(label="Quel est le montant du prêt que vous souhaitez obtenir ? (en €)"),
    gr.Number(label="Quel est la durée du prêt ? (en mois)"),
    gr.Radio(["Oui", "Non"], label="Avez-vous déjà remboursé un crédit ?"),
    gr.Radio(["Urbaine", "Semi-urbaine", "Rurale"], label="Dans quelle zone se situe le bien que vous souhaitez acquérir ?"),
]

outputs = [
    gr.Textbox(label="Décision du modèle"),
    gr.Textbox(label="Détail"),
]

title = "Prédiction d'éligibilité à un prêt bancaire"
description = (
    "Cette application utilise un modèle Random Forest entraîné sur le dataset de prêt "
    "pour prédire si un client est éligible à un prêt bancaire."
)

iface = gr.Interface(
    fn=predict_loan,
    inputs=inputs,
    outputs=outputs,
    title=title,
    description=description,
)

if __name__ == "__main__":
    iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f3e37548add0c85733.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
